## 프리트레인 마친 모델로 문장 생성

### 모델 로딩

- 프리트레인한 GPT2 모델과 토크나이저를 읽어들임
- KoGPT2는 부족한 한국어 성능을 극복하기 위해 40GB 이상의 텍스트로 학습된 한국어 디코더 언어모델

- 구글 드라이브 연동

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)  

Mounted at /gdrive


- 의존성 패키지 설치

In [2]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 61.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 106.2 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=58e5ab578805cc0df5dad5888d41abba27ba297f4bc2c77bf016aa0d63570609
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c

- 체크포인트 로드

In [3]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2",
)
model.eval()

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


- 토크나이저 로드

In [4]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


- 모델 입력값 만들기(언어모델에 넣을 프롬프트를 준비)

In [5]:
input_ids = tokenizer.encode("안녕하세요", return_tensors="pt")

In [6]:
input_ids

tensor([[25906,  8702,  7801,  8084]])

### Greedy search

- 매순간 최선(best)를 선택해 탐색 범위를 줄여보자는 것이 핵심 아이디어

- 다음 단어 확률 분포에서 최대 확률을 내는 단어들을 리턴

- do_sample: 'False'로 적용하면 여러 번 수행하더라도 생성 결과가 바뀌지 않음

- max_length: 생성 최대 길이
  - 이보다 길거나, 짧더라도 EOD 토큰 등 스페셜 토큰이 나타나면 생성을 중단

- min_length: 생성 최소 길이
  - 이보다 짧은 구간에서 EOD 등 스페셜 토큰이 등장해 생성이 중단될 경우 해당 토큰이 나올 확률을 0으로 수정하여 문장 생성이 종료되지 않도록 강제

In [8]:
import torch
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=False,
      min_length=10,
      max_length=50,
  )
  #토큰 인덱스를 문장으로 복원하기
  #그리디 서치를 수행한 결과인 generated_ids는 토큰 인덱스 시퀀스여서 알아보기 어렵기 때문
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



### Beam search

- 빔(beam) 크기만큼의 선택지를 계산 범위에 포함
- 다음 단어 확률 분포에서 `num_beams`만큼의 경우의 수를 남겨가면서 문장을 생성
- Beam search는 Greedy search보다 계산량이 많지만 좀 더 확률값이 높은 문장을 생성할 수 있음

In [9]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=False, #확률값이 높은 단어를 다음 단어로 결정
      min_length=10,
      max_length=50,
      num_beams=3, #빔 크기 변경
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그


- num_beams를 1로 입력하면 Greedy search와 똑같은 효과

In [10]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=False, #확률값이 높은 단어를 다음 단어로 결정
      min_length=10,
      max_length=50,
      num_beams=1, #빔 크기 변경
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



### 반복되는 표현 줄이기

#### n-gram 사이즈를 지정

- 위의 예시를 보면 `"그럼, 그건 뭐예요?"`이 반복됨
- 이를 아래와 같이 지정해 반복을 방지함
- 토큰이 n-gram 단위로 반복될 경우 모델이 계산한 결과를 무시하고 해당 n-gram의 마지막 토큰 등장 확률을 0으로 만들어 생성에서 배제
  * no_repeat_ngram_size:  n개 이상의 토큰이 반복될 경우 해당 n-gram 등장 확률을 0으로 만들어 생성 결과에서 배제함

In [11]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=False, 
      min_length=10,
      max_length=50,
      no_repeat_ngram_size=3,  #토큰 3개 이상 반복될 경우 3번재 토큰 확률 0으로 변경 
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?" 하고 나는 물었다.
"그건 뭐죠?" 나는 물었다.
나는 대답하지 않았다.
"그런데 왜 그걸 물어요? 그건 무슨 뜻이에요?


#### repetition penalty

- repetition_penalty라는 인자로 반복을 통제
- 그 값이 1.0보다 클수록 패널티가 세게 적용
- repetition_penalty값을 1.0으로 주면 아무런 패널티도 적용되지 않아 그리디 서치와 똑같은 효과

In [12]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=False, 
      min_length=10,
      max_length=50,
      repetition_penalty=1.5,  #리피티션 패널티 적용  
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요. 그리고 제가 할 수 있는 일은 아무것도 없어요.
이제 그만 돌아가고 싶어요.
제가 하는 일이 무엇


In [14]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=False, 
      min_length=10,
      max_length=50,
      repetition_penalty=1.2,    
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요."
"그래서 오늘은 제가 할 수 있는 일이 무엇인지 말해 보겠습니다."
"이제


In [13]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=False, 
      min_length=10,
      max_length=50,
      repetition_penalty=1.0,   
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



#### top-k sampling

- 모델이 예측한 다음 토큰 확률 분포 에서 확률값이 가장 높은  k 개 토큰 가운데 하나를 다음 토큰으로 선택하는 기법
- 다음 단어를 뽑을 때 확률값 기준 가장 큰 k개 가운데 하나를 선택하는 기법
  * 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, k개 안에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있음
  * 따라서 top-k sampling은 매 시행 때마다 생성 결과가 달라짐
  * k는 1 이상의 값

In [15]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=True,   #샘플링 방식으로 다음 토큰 생성
      min_length=10,
      max_length=50,
      top_k=50,     #확률값이 가장 높은 k개 토큰 가운데 하나를 다음 토큰으로 선택
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"(물론 그건 아니지만..)
"어차피 안 돼."(그냥 줘. 다행이야. ) 고만해야 하는 거 아니야? 뭘 잘해봐.
"마음껏


- top_k를 1로 입력하면 do_sample 인자를 True로 두더라도 그리디 서치와 똑같은 효과

In [16]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=True,   
      min_length=10,
      max_length=50,
      top_k=1,     
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



##### +temperature scaling

- top-k sampling은 temperature scaling과 동시에 적용할 수 있음 

- 모델의 다음 토큰 확률분포에 변형을 가해 문장을 다양하게 생성하는 기법
  - 확률분포를 변형한다는 의미는, 대소 관계의 역전 없이 분포의 모양만을 바꾼다는 의미
  - 이 값이 0에 가까울 수록 확률분포 모양이 원래 대비 뾰족해 짐
  - 순위의 변동은 없지만 원래 컸던 확률은 더 커지고, 작았던 확률은 더 작아져 확률분포의 모양이 뾰족(sharp)해짐
  - 그만큼 확률값 기준 1등 토큰이 다음 토큰으로 뽑힐 가능성이 높아짐
  
- temperature를 1보다 작게 하면 상대적으로 정확한 문장을, 1보다 크게 하면 상대적으로 다양한 문장을 생성

- 템퍼러처 스케일링은 탑k 샘플링, 탑p 샘플링과 같이 적용해야 의미가 있음

(1) t가 0에 가까워질 수록 토큰 분포가 sharp해짐 > 1등 토큰이 뽑힐 확률이 그만큼 높아짐 > do_sample=True이지만 사실상 greedy decoding이 됨

In [17]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=True,   
      min_length=10,
      max_length=50,
      top_k=50,
      temperature=0.5,     #탬퍼러처 스케일링 적용
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"라며 "제가 이 자리에 온지 1년이 넘었는데 아직도 그 자리에 서있는 걸 보니 정말 답답하다"고 말했다.
이어 "그동안 제가 할 수 있는 것은 다 해봤지만 이렇게까지 할 수 없는 게 도


(2) t=1이라면 모델 출력 분포를 그대로 사용, 하지만 샘플링 방식을 사용하기 때문에 생성할 때마다 다른 문장이 나옴

In [18]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=True,   
      min_length=10,
      max_length=50,
      top_k=50,
      temperature=1.0,     #탬퍼러처 스케일링 적용
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요’라는 질문에 "저희들은 내일까지 (지금은) 미뤄야 할 일이 많다"며 "늦어도 오늘 내로 마무리할 생각"이라고 말했다.
지난해 말 미국산 쇠고기 수입위생조건 협상, 그리고 한미 F


(3) t를 키울수록 토큰 분포가 uniform해짐 > 사실상 uniform sampling이 됨 > 그만큼 다양한 문장이 생성될 가능성이 높아지지만 생성 문장의 품질이 나빠질 수 있음

In [19]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=True,   
      min_length=10,
      max_length=50,
      top_k=50,
      temperature=100000000.0,     #탬퍼러처 스케일링 적용
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요', '소심한 너였다며'는 기자가 다시 찾아 왔으며 다음을 계획했다는 내용의 대필 청이 들어가 있었고 세번째 글에서는 댓가와 댓갈에 감사합습니닷. '저때까지만 안녕'이란 부고가


#### top-p sampling

- 다음 단어를 뽑을 때 누적 확률값이 p 이하인 단어들 가운데 하나를 선택하는 기법
  - 뉴클리어스 샘플링(necleus sampling)이라고도 불림
  - 확률값을 기준으로 단어들을 내림차순 정렬해 그 값이 높은 단어들을 후보로 삼는다는 점에서는 탑k 샘플링과 같지만 상위  k개를 후보로 삼느냐(탑k 샘플링), 누적 확률값이 p 이하인 단어들을 후보로 삼느냐(탑p 샘플링)에 따라 차이

- 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, 누적 확률값 p 이하에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있음
  - 따라서 top-p sampling은 매 시행 때마다 생성 결과가 달라짐 

- p는 확률이기 때문에 0~1 사이의 값 
  - p가 1이라면 어휘 집합에 있는 모든 단어를 대상으로 샘플링하기 때문에 top-p sampling 효과가 사라짐
  - p가 1보다 약간 작다면 확률값이 낮은 일부 단어들을 다음 단어 후보에서 제거해 생성 품질을 높임

- top_p를 1.0으로 설정한다면 확률값이 낮은 단어를 전혀 배제하지 않고 다음 단어 후보로 전체 어휘를 고려한다는 의미

In [20]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=True,   
      min_length=10,
      max_length=50,
      top_p=0.9,
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"내가 보기에 너무 잘난 것은 아닌가."
"그런 일은 없었어요?"
"그런 일은 없나요?"
"그런 일은 없나요?"
"그럼, 아무래도."
"


- p가 0에 가까울수록 후보 토큰이 줄어들어 확률값이 높은 토큰들만 남게 되어 그리디 서치와 비슷해짐

In [21]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=True,   
      min_length=10,
      max_length=50,
      top_p=0.01,
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



#### 종합 적용

In [23]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=True,   #샘플링 방식 적용
      min_length=10,     #문장 길이 설정
      max_length=50,
      repetition_penalty=1.5,   #반복 줄이기
      no_repeat_ngram_size=3,
      temperature=0.9,      #탬퍼러처 스케일링
      top_k=50,
      top_p=0.92,
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?' 라는 말로 대답했습니다.
이런 질문에는 씁쓸한 기분이 들었다고 합니다.
아무튼 '어떤 사람이 좋은 것만을 말해서 모든 것을 좋게 해 주려고 한다면 나쁜 것이 아닐까?', '어디선
